In [1]:
PROJECT_ROOT_PATH = "/workdir/DockerML_sandbox/"

datapath = PROJECT_ROOT_PATH + "lab_competition/data/"
outpath = PROJECT_ROOT_PATH + "lab_competition/output/01/"


# sudachiの小さい辞書をインポート
!pip install pyproject-toml
!pip install sudachipy sudachidict_core
!pip install scikit-learn

import numpy as np
import collections

In [2]:
# 各データを読み込みリストに格納
def read_file(path):
    with open(path, mode="r") as f:
        result = f.read().splitlines()
    return result

train_text = read_file(datapath + "text.train.txt")
dev_text = read_file(datapath + "text.dev.txt")
test_text = read_file(datapath + "text.test.txt")
train_label = np.loadtxt(datapath + "label.train.txt")
dev_label = np.loadtxt(datapath + "label.dev.txt")

In [3]:
from sudachipy import tokenizer
from sudachipy import dictionary
from itertools import chain

tokenizer_obj = dictionary.Dictionary().create()
mode = tokenizer.Tokenizer.SplitMode.C

# 前処理
def text_cleaning(textlist, mode, clear_part_of_speech_list):
    morphemelist = [tokenizer_obj.tokenize(text, mode) for text in textlist]
    result = []
    for morpheme in morphemelist:
        words = []
        for word in morpheme:
            if word.part_of_speech()[0] not in clear_part_of_speech_list:
                words.append(word.normalized_form())
        result.append(" ".join(words))
    return result

In [29]:
clear_part_of_speech_list = ["補助記号", "助詞","名詞"]

train_data = text_cleaning(train_text, mode, clear_part_of_speech_list)
dev_data = text_cleaning(dev_text, mode, clear_part_of_speech_list)
test_data = text_cleaning(test_text, mode, clear_part_of_speech_list)

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(2,2))
train_tfidf = vectorizer.fit_transform(train_data)
dev_tfidf = vectorizer.transform(dev_data)

train_vec = train_tfidf.toarray()
dev_vec = dev_tfidf.toarray()

print("train = " + str(train_vec.shape))
print("trainlabel = " + str(train_label.shape))
print("dev = " + str(dev_vec.shape))
print("devlabel = " + str(dev_label.shape))

train = (30000, 56335)
trainlabel = (30000,)
dev = (2500, 56335)
devlabel = (2500,)


# 学習

In [40]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score

model = MultinomialNB()
model.fit(train_vec, train_label)

dev_pred = model.predict(dev_vec)
acc = accuracy_score(dev_pred, dev_label)
kappa = cohen_kappa_score(dev_pred, dev_label)
print("正確率 = %.3f" % (acc))
print("QuadraticWeightedKappa = %.3f" % (acc))
print(classification_report(dev_label, dev_pred))

正確率 = 0.296
QuadraticWeightedKappa = 0.296
              precision    recall  f1-score   support

        -2.0       0.50      0.00      0.01       310
        -1.0       0.16      0.02      0.04       415
         0.0       0.27      0.83      0.41       647
         1.0       0.43      0.22      0.29       837
         2.0       0.33      0.00      0.01       291

    accuracy                           0.30      2500
   macro avg       0.34      0.22      0.15      2500
weighted avg       0.34      0.30      0.21      2500

